In [50]:
import json
record_path = "/dataset/zhuotaotian/bhpeng/vectordb/output/record.json"
config_path = "/dataset/zhuotaotian/bhpeng/vectordb/output/config_6*20_all.json"
index_range = {
        "ag": [
            0,
            4
        ],
        "amazon": [
            4,
            8
        ],
        "dbpedia": [
            8,
            12
        ],
        "yahoo": [
            12,
            16
        ],
        "yelp": [
            16,
            20
        ]
    }
tasks = ['ag', 'amazon', 'dbpedia', 'yahoo', 'yelp']
topk = 2
groups = 6

for k, v in index_range.items():
    index_range[k] = list(range(v[0], v[1]))
index_range

{'ag': [0, 1, 2, 3],
 'amazon': [4, 5, 6, 7],
 'dbpedia': [8, 9, 10, 11],
 'yahoo': [12, 13, 14, 15],
 'yelp': [16, 17, 18, 19]}

In [51]:
with open(record_path) as f:
    data = json.load(f)
print(data['ag'][0], len(data['ag']))

{'group': 0, 'index': 1, 'label': 'ag'} 91200


In [62]:
def compute_accuracy(data, thresh=0.5):
    accuracy = dict()
    for task in tasks:
        accuracy[task] = dict(total_num=0, total_right=0, single_right_rate=0)
        single_right = 0
        for i, d in enumerate(data[task]):
            accuracy[task]['total_num'] += 1
            if d['index'] in index_range[task]:
                accuracy[task]['total_right'] += 1
                single_right += 1
            if i % (topk*groups) == (topk * groups - 1):
                if single_right / topk / groups > thresh:
                    accuracy[task]['single_right_rate'] += 1
                single_right = 0
        accuracy[task]['total_accuracy'] = accuracy[task]['total_right'] / accuracy[task]['total_num']
        accuracy[task]['single_right_rate'] /= (len(data[task]) / groups / topk)
    return accuracy
accuracy = compute_accuracy(data)
for attr in ['total_accuracy', 'single_right_rate']:
    print(attr+":")
    s=0
    for task in tasks:
        print("  ",  "{:7s}: ".format(task), accuracy[task][attr])
        s+=float(accuracy[task][attr])
    print("  ",  "{:7s}: ".format('avg'), s/5)

total_accuracy:
   ag     :  0.8687061403508772
   amazon :  0.8682785087719298
   dbpedia:  0.8796271929824562
   yahoo  :  0.7445504385964913
   yelp   :  0.9233881578947368
   avg    :  0.8569100877192983
single_right_rate:
   ag     :  0.98
   amazon :  0.9801315789473685
   dbpedia:  0.9857894736842105
   yahoo  :  0.8630263157894736
   yelp   :  0.9952631578947368
   avg    :  0.9608421052631579


{'ag': {'num': 91200, 'right': 79172, 'accuracy': 0.8681140350877193},
 'amazon': {'num': 91200, 'right': 78973, 'accuracy': 0.8659320175438596},
 'dbpedia': {'num': 91200, 'right': 80645, 'accuracy': 0.8842653508771929},
 'yahoo': {'num': 91200, 'right': 67997, 'accuracy': 0.7455811403508772},
 'yelp': {'num': 91200, 'right': 83723, 'accuracy': 0.918015350877193}}